## Release of apipie-rails gem

### Requirements
- push access to https://github.com/Apipie/apipie-rails
- push access to rubygems.org for apipie-rails
- sudo yum install python-slugify asciidoc
- ensure neither the `git push` or `gem push` don't require interactive auth. If you can't use api key or ssh key to auth skip these steps and run them form the shell manually
- ensure all checks have passed on the branch you're about to release

### Release process
- Follow the steps with `<Shift>+<Enter>` or `<Ctrl>+<Enter>,<Down>`
- If anything fails, fix it and re-run the step if applicable

### Release settings

In [ ]:
%autosave 0

In [ ]:
%cd ..

### Update the following notebook settings

In [ ]:
NEW_VERSION = '0.5.20'
LAST_VERSION = '0.5.19'
GIT_REMOTE_UPSTREAM = 'origin'
STABLE_RELEASE = False
WORK_BRANCH = 'stable' if STABLE_RELEASE else 'master'
# Array of strings, e.g. ["21cbsc214g3", "21casc214g3"]
CHERRY_PICKS = []
GEMFILE='Gemfile.rails61'

### Ensure the repo is up to date

In [ ]:
! git checkout {WORK_BRANCH}

In [ ]:
! git fetch {GIT_REMOTE_UPSTREAM}

In [ ]:
! git rebase {GIT_REMOTE_UPSTREAM}/{WORK_BRANCH}

### Cherry picks for stable release

In [ ]:
if STABLE_RELEASE:
    for cp in CHERRY_PICKS:
        ! git cherry-pick -x {cp}

### Run tests locally if your setup allows, otherwise ensure the HEAD is green

In [ ]:
! BUNDLE_GEMFILE=gemfiles/{GEMFILE} bundle update

In [ ]:
! BUNDLE_GEMFILE=gemfiles/{GEMFILE} bundle exec rspec

### Update release related stuff

In [ ]:
! sed -i 's/VERSION = .*/VERSION = "{NEW_VERSION}"/' lib/apipie/version.rb

In [ ]:
# Parse git changelog
from IPython.display import Markdown as md
from subprocess import check_output
from shlex import split
import re

def format_log_entry(entry):
    author = re.search(r'author:(.*)', entry).group(1)
    entry = re.sub(r'author:(.*)', '', entry)
    entry = re.sub(r'([fF]ixes|[rR]efs)[^-]*-\s*(.*)', r'\2', entry)
    entry = '* ' + entry.capitalize()
    entry = re.sub(r'\(#([0-9]+)\)', r'[#\1](https://github.com/Apipie/apipie-rails/pull/\1)', entry)
    entry = entry + f'({author})'
    return entry

def skip(entry):
    if re.match(r'Merge pull', entry) or \
      re.match(r'^i18n', entry) or \
      re.match(r'^Bump to version', entry):
        return True
    else:
        return False 
git_log_cmd = 'git log --pretty=format:"%%s author:%%an" v%s..HEAD' % LAST_VERSION
log = check_output(split(git_log_cmd)).decode('utf8').split('\n')
change_log = [format_log_entry(e) for e in log if not skip(e)]
md('\n'.join(change_log))


In [ ]:
# Write release notes
from datetime import datetime
import fileinput
import sys

fh = fileinput.input('CHANGELOG.md', inplace=True)  
for line in fh:  
    print(line.rstrip())
    if re.match(r'========', line):
        print('## [v%s](https://github.com/Apipie/apipie-rails/tree/v%s) (%s)' % (NEW_VERSION, NEW_VERSION, datetime.today().strftime('%Y-%m-%d')))
        print('[Full Changelog](https://github.com/Apipie/apipie-rails/compare/v%s...v%s)' % (LAST_VERSION, NEW_VERSION))
        for entry in change_log:
            print(entry)
        print('')
fh.close()  

#### Manual step: Update deps in the gemspec if necessary

### Check what is going to be committed

In [ ]:
! git add -u
! git status

In [ ]:
! git diff --cached

### Commit changes

In [ ]:
! git commit -m "Bump to {NEW_VERSION}"

### Tag new version

In [ ]:
! git tag {NEW_VERSION}

### Build the gem

In [ ]:
! BUNDLE_GEMFILE=gemfiles/{GEMFILE} bundle exec rake build

In [ ]:
! gem push pkg/apipie-rails-{NEW_VERSION}.gem

### PUSH the changes upstream If everything is correct

In [ ]:
! git push {GIT_REMOTE_UPSTREAM} {WORK_BRANCH}

In [ ]:
! git push --tags {GIT_REMOTE_UPSTREAM} {WORK_BRANCH}

#### Now the new release is in upstream repo

### Some manual steps follow to improve the UX

#### New release on GitHub

Copy the following changelog lines to the description in form on link below
The release title is the new version.

In [ ]:
print('\n')
print('\n'.join(change_log))
print('\n\nhttps://github.com/Apipie/apipie-rails/releases/new?tag=%s' % NEW_VERSION)

## Congratulations

Release is public now.